<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/SwissLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# upload HDFS dataset
!wget 'https://zenodo.org/record/3227177/files/HDFS_1.tar.gz'
!tar -xzvf "/content/HDFS_1.tar.gz" -C "/content/"   #unzip the file

--2023-08-04 15:12:25--  https://zenodo.org/record/3227177/files/HDFS_1.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 161886385 (154M) [application/octet-stream]
Saving to: ‘HDFS_1.tar.gz’

HDFS_1.tar.gz       100%[===================>] 154.39M   595KB/s    in 4m 33s  

2023-08-04 15:17:00 (580 KB/s) - ‘HDFS_1.tar.gz’ saved [161886385/161886385]

HDFS.log
anomaly_label.csv


In [3]:
with open('/content/HDFS.log', 'r') as input_fp, open('/content/first_1_million_lines.log', 'w') as output_fp:
    for i, line in enumerate(input_fp):
        if i >= 1000000:
            break
        output_fp.write(line)

In [2]:
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/EngCorpus.pkl'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/tokenize_group_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/mask_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/knowledge_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/file_output_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/dict_group_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/evaluator/evaluator.py'


--2023-08-04 15:17:11--  https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/EngCorpus.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 412351 (403K) [text/plain]
Saving to: ‘EngCorpus.pkl’

EngCorpus.pkl       100%[===================>] 402.69K  --.-KB/s    in 0.04s   

2023-08-04 15:17:11 (9.72 MB/s) - ‘EngCorpus.pkl’ saved [412351/412351]

--2023-08-04 15:17:12--  https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/tokenize_group_layer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiti

In [4]:
# upload logs
!wget 'https://raw.githubusercontent.com/logpai/loghub/master/HDFS/HDFS_2k.log'

--2023-08-04 15:17:56--  https://raw.githubusercontent.com/logpai/loghub/master/HDFS/HDFS_2k.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 285848 (279K) [text/plain]
Saving to: ‘HDFS_2k.log’

HDFS_2k.log         100%[===================>] 279.15K  --.-KB/s    in 0.04s   

2023-08-04 15:17:56 (7.14 MB/s) - ‘HDFS_2k.log’ saved [285848/285848]



In [5]:
import sys
sys.path.append('/content/') #to be able for importing libarary from this address

In [6]:
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541529 sha256=5e055c138fa34e05577f89264da0888457a4bb97084bf4b6cceb0db968a9d156
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja


In [7]:
from file_output_layer import FileOutputLayer
from knowledge_layer import KnowledgeGroupLayer
from mask_layer import MaskLayer
from tokenize_group_layer import TokenizeGroupLayer
from dict_group_layer import DictGroupLayer
import evaluator

import os
import re
import string
import hashlib
from datetime import datetime
from tqdm import tqdm
import pandas as pd

In [ ]:
input_dir = '/content/' # The input directory of log file
output_dir = 'LogParserResult/' # The output directory of parsing results

In [ ]:
log_file = 'HDFS_2k.log'
log_format =  '<Date> <Time> <Pid> <Level> <Component>: <Content>'
regex = [r'blk_-?\d+', r'(\d+\.){3}\d+(:\d+)?']


In [8]:
setting = {
        'log_file': 'HDFS_2k.log',
        'log_format': '<Date> <Time> <Pid> <Level> <Component>: <Content>',
        'regex': [r'blk_-?\d+', r'(\d+\.){3}\d+(:\d+)?']
        }

In [9]:
def generate_logformat_regex(logformat):
    """ Function to generate regular expression to split log messages
    """
    headers = []
    splitters = re.split(r'(<[^<>]+>)', logformat)
    regex = ''
    for k in range(len(splitters)):
        if k % 2 == 0:
            splitter = re.sub(' +', '\\\s+', splitters[k])
            regex += splitter
        else:
            header = splitters[k].strip('<').strip('>')
            regex += '(?P<%s>.*?)' % header
            headers.append(header)
    regex = re.compile('^' + regex + '$')
    return headers, regex



In [10]:
def load_logs(log_file, regex, headers):
    """ Function to transform log file to dataframe
    """
    log_messages = dict()
    linecount = 0
    with open(log_file, 'r') as fin:
        for line in tqdm(fin.readlines(), desc='load data'):

            try:
                linecount += 1
                match = regex.search(line.strip())
                message = dict()
                for header in headers:
                    message[header] = match.group(header)
                message['LineId'] = linecount
                log_messages[linecount] = message
            except Exception as e:
                pass
    return log_messages

In [11]:
headers, regex = generate_logformat_regex(setting['log_format'])
log_messages = load_logs('/content/first_1_million_lines.log', regex, headers)
# preprocess layer
log_messages = KnowledgeGroupLayer(log_messages).run()
# tokenize layer
log_messages = TokenizeGroupLayer(log_messages, rex=setting['regex']).run()
# dictionarize layer and cluster by wordset
dict_group_result = DictGroupLayer(log_messages, '/content/EngCorpus.pkl').run()
# apply LCS and prefix tree
results, templates = MaskLayer(dict_group_result).run()

priori knowledge preprocess: 100%|██████████| 1000000/1000000 [00:27<00:00, 36349.21it/s]


Knowledge group layer finished.


tokenization: 100%|██████████| 1000000/1000000 [00:30<00:00, 33115.07it/s]


Tokenization layer finished.


group by wordset: 100%|██████████| 1000000/1000000 [00:01<00:00, 825000.95it/s]


After Dictionary Group, total: 35 bin(s)


generate output: 100%|██████████| 35/35 [00:03<00:00,  9.64it/s]

After mask layer finish, tot, total: 35 bin(s)


In [12]:
FileOutputLayer(log_messages, '/content/results/HDFS', templates, ['LineId'] + headers).run()

In [13]:
import pickle

with open('/content/templates.pkl', 'rb') as file:
    data = pickle.load(file)

In [15]:
data[0]

['Receiving', 'block', '<*>', 'src', ':', '/', '<*>', 'dest', ':', '/', '<*>']